In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

# import python CRUD module for MongoDB
import mongocrud

###########################
# Data Manipulation / Model
###########################
# Create connection using mongocrud

address = '127.0.0.1'
username = 'aacuser'
password = 'badPasswd'
port = 52879
database = 'AAC'
collection = 'animals'

shelter = mongocrud.DatabaseConnector(address,
                                      username,
                                      password, 
                                      port,
                                      database,
                                      collection)
query = shelter.read({'animal_type':'Cat'})

# class read method must support return of cursor object and accept projection json input
# read all dogs from database collection as all rescue animals for this project are dogs
df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}))

# move animal type and breed to second and third columns to make more easily accessible
df.insert(2,'name',df.pop('name'))
df.insert(2,'outcome_type',df.pop('outcome_type'))
df.insert(1,'breed',df.pop('breed'))
df.insert(1,'animal_type',df.pop('animal_type'))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# define color for active and selected cells
style_data_conditional = [
    {
        "if": {"state": "active"},
        # shades of green
        "backgroundColor": "rgba(150, 255, 180, 0.2)",
        "border": "1px solid green",
    },
]

# define logo image location
image_filename = 'Grazioso Salvare Logo.png'
# load logo image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# define app layout
app.layout = html.Div([
    # center map, pie chart, and logo at top
    html.Div(className='row',
             style={'display' : 'flex','align-items':'center','justify-content':'center'},
             children=[
                 html.Div(
                        id='map-id',
                        className='col s12 m6',
                 ),
             dcc.Graph(id="pie-chart-id"),
                 
             # logo is also an anchor to developing company's website
             html.A([
                     html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                              style={'height':'80%','width':'80%'}),
                 ], href='http://www.snhu.edu'
                  , target='_blank')
             ],
        ),
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # personal identifier
    html.Center(html.B(html.H1('David Smith SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    # add buttons to filter the data. Buttons are left to right and centered
    # buttons were chosen to allow sorting with fewer required clicks vs a drop down list
    html.Div(className='row',
             style={'display' : 'flex','align-items':'center','justify-content':'center'},
                  children=[
                          # each button will filter based on the children string
                          html.Button(id='submit-button-one', n_clicks=0, children='Water'),
                          html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness'),
                          html.Button(id='submit-button-three', n_clicks=0, children='Disaster or Individual Tracking'),
                          html.Button(id='submit-button-four', n_clicks=0, children='Reset'),
                  ]),
    
    # add datatable to show database information
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns if i != "id"
        ],
        # load data into datatable from dataframe
        data=df.to_dict('records'),
        editable=False,
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        page_action="native",
        page_current=0,
        # limit number of results to twenty per page
        page_size=20,
        style_data_conditional=style_data_conditional,
    ),
    html.Br(),
    html.Hr(),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Highlight row of currently clicked cell
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "active_cell")]
)
def update_selected_row_color(active):
    style = style_data_conditional.copy()
    if active:
        # set style to change color for current active cell's entire row
        style.append(
            {
                "if": {"row_index": active["row"]},
                "backgroundColor": "rgba(150, 255, 180, 0.2)",
                "border": "1px solid green",
            },
        )
    return style

# Update map marker and center for currently active cell's data
@app.callback(
    [Output('map-id', "children"),
     Output('pie-chart-id', "figure"),],
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "active_cell")],
    [State('datatable-id', "children"),
    ])
def update_display(viewData, active_cell,datatable_state):
    
    # create dataframe with current datatable view's data
    dff = pd.DataFrame.from_dict(viewData)
    
    # if the view is empty, don't update the datatable
    if dff.size == 0:
        return datatable_state
    
    # if no cell has been clicked, choose the first row automatically. This also avoids index out of bounds when filtering
    if not active_cell:
        active_row = 0
    
    # if a cell has been clicked, use its row
    else:
        active_row = active_cell["row"]
    
    # get location coordinates for current animal
    location = [dff.iloc[active_row,dff.columns.get_loc("location_lat")], dff.iloc[active_row,dff.columns.get_loc("location_long")]]
    
    # convert coordinates for use in marker label
    location_string = location.copy()
    location_string.insert(1, ", ")
    
    # create pie chart from current view
    pie_chart = px.pie(data_frame=dff,
                       names='breed',
                       title='Dog Breeds On Current Table Page',
                       color_discrete_sequence=px.colors.qualitative.Set3,
                       )
    pie_chart.update_layout(title={'x':0.5})
    
    # return data for map mmarker and pie chart
    return [
        # center map on marker
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[location[0],location[1]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # marker with tooltip and popup
            dl.Marker(position=location, children=[
                # mouseover of marker set to animal_id
                dl.Tooltip(str(dff.iloc[active_row,dff.columns.get_loc("animal_id")])),
                dl.Popup([
                    # display name and breed of animal
                    html.Center(html.H1((dff.iloc[active_row,dff.columns.get_loc("name")] or "Unnamed"))),
                    html.Center(html.H2((dff.iloc[active_row,dff.columns.get_loc("breed")]))),
                    # display location coordinates
                    html.P(location_string)
                ])
            ])
        ]), pie_chart
    ]

# Filter datatable based on rescue type
@app.callback([Output('datatable-id',"data"),
               Output('datatable-id',"active_cell"),
               Output('datatable-id',"selected_cells")],
              [Input('submit-button-one', 'n_clicks'),
               Input('submit-button-two', 'n_clicks'),
               Input('submit-button-three', 'n_clicks'),
               Input('submit-button-four', 'n_clicks'),
              ],
               State('datatable-id',"children"))
def on_click(bt1,bt2,bt3,bt4,current_state):
    
    # if no button has been clicked, initialize datatable
    if not dash.callback_context.triggered:
    
        # choose all dogs from database
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}))
        
        # return dataframe to datatable, selecting first row and first colummn
        return df.to_dict('records'), {"row":0, "column":0}, {"row":0, "column":0}
    
    # check which button was clicked
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    # if Reset was clicked
    if 'submit-button-four' in changed_id or not changed_id:
    
        # show all dogs
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}))
    else:
        # if Water was clicked
        if 'submit-button-one' in changed_id:
            # show Water results
            rescue_type = "water"
    
        # if Mountain or Wilderness was clicked
        elif 'submit-button-two' in changed_id:
            # show Mountain or Wilderness results
            rescue_type = "mountain_wilderness"
        
        # if Disaster or Individual Tracking was clicked
        elif 'submit-button-three' in changed_id:
            # show Disaster or Individual Tracking results
            rescue_type = "disaster_individual_tracking"
        else:
            # if no button was pressed and not initializing, do not update datatable
            return current_state
        
        # define dogs described for this category
        breeds, max_age, min_age, preferred_sex = query_information(rescue_type)
        
        # retrieve data fromm database
        df = pd.DataFrame.from_records(shelter.read(
             {"$and":[
                 {'breed':{"$in":breeds}},
                 {"age_upon_outcome_in_weeks":{"$lt":max_age,"$gt":min_age}},
                 {"sex_upon_outcome":{"$eq":preferred_sex}}
             ]}))
        
    # return results, defaulting to select first row and first column
    return df.to_dict('records'), {"row":0, "column":0}, {"row":0, "column":0}


# get selected rescue type's preferred breeds, max_age, min_age, and preferred sex
def query_information(rescue_type):
    
    # preferred_sex based on mountain_wilderness and disaster_invdividual_tracking having same value
    preferred_sex = "Intact Male"
    
    # age limits in weeks based on water and mountain_wilderness having same values
    min_age = 26
    max_age = 156
    
    # change data for each type of rescue
    if rescue_type == "water":
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        preferred_sex = "Intact Female"
    if rescue_type == "mountain_wilderness":
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
    if rescue_type == "disaster_individual_tracking":
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        min_age = 20
        max_age = 300
    
    # return desired information
    return breeds, max_age, min_age, preferred_sex

# run app
app